# 📘 Tutorial: Using Evidently Regression Presets with Tests

In [ ]:
import pandas as pd
from sklearn import datasets, model_selection, linear_model

from evidently import Report, DataDefinition, Dataset, Regression
from evidently.presets import RegressionDummyQuality, RegressionQuality, RegressionPreset
from evidently.tests import lte, gte, lt, gt, is_in, not_in, eq, not_eq
from evidently.core.metric_types import MeanStdMetricTests

# Regression presets

## 🧩 1. What Are Regression Presets?

Evidently provides pre-defined “presets” for common testing scenarios in regression tasks. These presets help evaluate model behavior using curated sets of metrics and test conditions — so you can focus on insights, not boilerplate.

In [ ]:
housing_data = datasets.fetch_california_housing(as_frame=True)

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(housing_data.data, 
                                                                    housing_data.target, 
                                                                    test_size=0.4, 
                                                                    random_state=42)

In [ ]:
features = X_train.columns.to_list()

In [ ]:
model = linear_model.Ridge(random_state=42)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
preds_train = model.predict(X_train)
preds_test = model.predict(X_test)

In [ ]:
X_train['target'] = y_train
X_train['prediction'] = preds_train

In [ ]:
X_test['target'] = y_test
X_test['prediction'] = preds_test

In [ ]:
data_definition=DataDefinition(
        regression=[Regression()], #or you can specify parameters Regression(target='target', prediction='prediction')
        numerical_columns=features 
    )

In [ ]:
reference = Dataset.from_pandas(
    pd.DataFrame(X_train),
    data_definition=data_definition,
)

current = Dataset.from_pandas(
    pd.DataFrame(X_test),
    data_definition=data_definition,
)

## Regression preset

In [ ]:
regression_preset = Report(metrics=[
    RegressionPreset()
    ],
    include_tests=True
)

In [ ]:
regression_report_no_ref = regression_preset.run(reference_data=None, current_data=current)

In [ ]:
regression_report_no_ref

In [ ]:
regression_report = regression_preset.run(reference_data=reference, current_data=current)

In [ ]:
regression_report

In [ ]:
regression_preset = Report(metrics=[
    RegressionPreset(
        mae_tests=[lt(0.3)],
        mean_error_tests=[gt(-0.2), lt(0.2)],
        mape_tests=[lt(20)],
        rmse_tests=[lt(0.3)],
        abs_max_error_tests=[lt(10)],
        r2score_tests=[gt(0.5)],
    )
])

In [ ]:
regression_report = regression_preset.run(reference_data=reference, current_data=current)

In [ ]:
regression_report

In [ ]:
regression_preset = Report(metrics=[
    RegressionPreset(
        mae_tests=MeanStdMetricTests(mean=[lt(0.2)], std=[lt(0.4)]), 
        mean_error_tests=MeanStdMetricTests(mean=[lt(0.2)], std=[lt(0.4)]) 
    )
])


In [ ]:
regression_report = regression_preset.run(reference_data=reference, current_data=current)

In [ ]:
regression_report

## Regression quality

In [ ]:
quality_preset = Report(metrics=[
    RegressionQuality(),
    ],
    include_tests=True
)

In [ ]:
quality_report_no_ref = quality_preset.run(reference_data=None, current_data=current)

In [ ]:
quality_report_no_ref

In [ ]:
quality_report = quality_preset.run(reference_data=reference, current_data=current)

In [ ]:
quality_report

In [ ]:
quality_preset = Report(metrics=[
    RegressionQuality(
        mean_error_tests=[lt(0.1)], 
        mae_tests=[lt(0.5)], 
        mape_tests=[lt(20)],
        rmse_tests=[lt(0.5)],
    )
])

In [ ]:
quality_report_with_metrics = quality_preset.run(reference_data=reference, current_data=current)

In [ ]:
quality_report_with_metrics

In [ ]:
quality_preset = Report(metrics=[
    RegressionQuality(
        mean_error_tests=MeanStdMetricTests(mean=[lt(0.2)], std=[lt(0.4)])
    )
])

In [ ]:
quality_report_mean_std_tests = quality_preset.run(reference_data=reference, current_data=current)

In [ ]:
quality_report_mean_std_tests

## Regression dummy quality

In [ ]:
dummy_quality_preset = Report(metrics=[
    RegressionDummyQuality(),
    ],
    include_tests=True
)

In [ ]:
dummy_quality_report_no_ref = dummy_quality_preset.run(reference_data=None, current_data=current)

In [ ]:
dummy_quality_report_no_ref

In [ ]:
dummy_quality_report = dummy_quality_preset.run(reference_data=reference, current_data=current)

In [ ]:
dummy_quality_report

In [ ]:
dummy_quality_preset = Report(metrics=[
    RegressionDummyQuality(
        mae_tests=[lt(0.5)], 
        mape_tests=[lt(20)],
        rmse_tests=[lt(0.5)],
        #max_error_test to be implemented
    )],
)

In [ ]:
dummy_quality_report = dummy_quality_preset.run(reference_data=reference, current_data=current)

In [ ]:
dummy_quality_report